# Calculate expectation value of collection order

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

## Import data structures

In [2]:
# Load the cost structure
with open('costos.json', 'r') as f:
    costos = json.load(f)

# Load the rate of success structure
with open('tasas_banco_metodo.json', 'r') as f:
    tasas = json.load(f)


In [ ]:
# Function to evaluate if an order is potentially profitable
def es_potencial(row):
    monto = row['montoCobrar']
    emisora_info = row['nombreEmisora'].split(maxsplit=1)
    if len(emisora_info) < 2:
        emisora_info = [row['nombreBanco'], 'default']
    banco, emisora = emisora_info
    metodo = "default"
    
    try:
        costo = costos[banco][metodo]
        tasa = tasas[banco][emisora]
        expected_value = tasa * monto
        return expected_value > costo
    except KeyError:
        # If no matching cost found, return None or False
        return None


In [4]:
# Dataframes complementarios (Traducción de IDs)                                         # 1ra Columna   -> Resto de columnas
df_id_emisora = pd.read_csv("data/ExtraccionDomiVersionFinal/CatEmisora.csv")            # ID Emisora    -> Nombre Emisora, ID Banco, Emisora??, Tipo envío
df_id_listaemisor = pd.read_csv("data/ExtraccionDomiVersionFinal/ListaCobroEmisora.csv") # ID ListaCobro -> ID Emisora
df_id_banco = pd.read_csv("data/ExtraccionDomiVersionFinal/CatBanco.csv")                # ID Banco      -> Nombre del banco

# Dataframes detallados por año
df_2024 = pd.read_csv("data/ExtraccionDomiVersionFinal/ListaCobroDetalle2024.csv")
df_2025 = pd.read_csv("data/ExtraccionDomiVersionFinal/ListaCobroDetalle2025.csv")


In [5]:
cobros_detallados = pd.concat([df_2024, df_2025], ignore_index=True)


In [6]:
# Standardize column names
df_id_banco = df_id_banco.rename(columns={'IdBanco': 'idBanco', 'Nombre': 'nombreBanco'})
df_id_emisora = df_id_emisora.rename(columns={'Nombre': 'nombreEmisora'})

# Drop duplicated columns
df_id_emisora = df_id_emisora.drop('IdBanco', axis=1)

# Standardize emisora entries
df_id_emisora['nombreEmisora'] = df_id_emisora['nombreEmisora'].str.strip()
df_id_emisora['nombreEmisora'] = df_id_emisora['nombreEmisora'].str.upper()

In [7]:
cobros_detallados = cobros_detallados.merge(df_id_banco, on='idBanco', how='left') # Obtener nombre de banco
cobros_detallados = cobros_detallados.merge(df_id_listaemisor, on='idListaCobro', how='left') # Obtener ID emisora
cobros_detallados = cobros_detallados.merge(df_id_emisora, on='idEmisora', how='left') # Obtener datos de emisora
cobros_detallados.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,nombreBanco,idEmisora,nombreEmisora,Emisora,TipoEnvio
0,139345,85784,35530241,12,539.53,539.53,0.0,NaN,4.0,BBVA MEXICO,6,BBVA TRADICIONAL REINTENTO,REINTENTO,TRADICIONAL
1,139345,122162,35530242,12,468.56,468.56,0.0,NaN,4.0,BBVA MEXICO,6,BBVA TRADICIONAL REINTENTO,REINTENTO,TRADICIONAL
2,139345,122162,35530243,12,780.94,780.94,0.0,NaN,4.0,BBVA MEXICO,6,BBVA TRADICIONAL REINTENTO,REINTENTO,TRADICIONAL
3,139345,122162,35530244,12,312.37,312.37,0.0,NaN,4.0,BBVA MEXICO,6,BBVA TRADICIONAL REINTENTO,REINTENTO,TRADICIONAL
4,139345,123763,35530245,12,817.48,817.48,0.0,NaN,4.0,BBVA MEXICO,6,BBVA TRADICIONAL REINTENTO,REINTENTO,TRADICIONAL


In [8]:
# Success flag
cobros_detallados['pagado'] = cobros_detallados['montoCobrado'] > 0

# Apply the evaluation
cobros_detallados['potencial'] = cobros_detallados.apply(es_potencial, axis=1)

## Test de hipótesis
$H_0$: Mean montoCobrado is the same across groups.

$H_1$: Mean montoCobrado is higher for potencial == True.

In [10]:
from scipy.stats import norm

# Count successes and total for each group
success_true = cobros_detallados[cobros_detallados['potencial'] == True]['pagado'].sum()
n_true = cobros_detallados[cobros_detallados['potencial'] == True]['pagado'].count()

success_false = cobros_detallados[cobros_detallados['potencial'] != True]['pagado'].sum()
n_false = cobros_detallados[cobros_detallados['potencial'] != True]['pagado'].count()

# Proportions
p1 = success_true / n_true
p2 = success_false / n_false

# Pooled proportion
p_pool = (success_true + success_false) / (n_true + n_false)

# Standard error
se = np.sqrt(p_pool * (1 - p_pool) * (1/n_true + 1/n_false))

# Z-score
z = (p1 - p2) / se

# Two-tailed p-value
pval = 2 * norm.sf(abs(z))

print(f"Z = {z:.4f}, p-value = {pval:.4f}")



Z = 288.5667, p-value = 0.0000


In [11]:
from scipy.stats import ttest_ind

group_true = cobros_detallados[cobros_detallados['potencial'] == True]['montoCobrado']
group_false = cobros_detallados[cobros_detallados['potencial'] != True]['montoCobrado']

# Perform t-test
t_stat, pval = ttest_ind(group_true, group_false, equal_var=False)  # Welch's t-test
print(f"T-test p-value: {pval:.4f}")


T-test p-value: 0.0000
